# Importar librerias

In [1]:
# !pip install scikit-learn
# !pip install matplotlib
!pip install seaborn



In [16]:
%load_ext autoreload
%autoreload 2

import dgl
import torch
from torch.optim import Adam
import itertools
import torch.nn.functional as F
import tqdm


from modules.gnn import GNN
from modules.predictors import DotPredictor, MLPPredictor
from modules.models import  ModelGraphSAGE, GraphSAGE, GCN, ModelGCN, ModelSAGESample #ModelGraphSAGE,GraphSAGE, GCN,ModelSAGESample,SAGE, ModelGCN
from utils import plot_roc_curve, plot_training, calculate_metrics

import pandas as pd 
import numpy as np

# Reload modulos automaticamente
# %load_ext autoreload
# %autoreload 2
# from modules.graph_from_api import Graph_API
# from dgl.sampling import pack_traces


# from torch.optim import SparseAdam

import matplotlib.pyplot as plt

# from modules.graph import Graph
# import numpy as np
import os
# import fnmatch
# import tqdm
# import networkx as nx

# import dgl.function as fn
# from dgl.nn import DeepWalk
# from torch.utils.data import DataLoader

# from sklearn.linear_model import LogisticRegression
# from gensim.models import Word2Vec
# from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
ls data/DGL_Graph/DiGraph_AllFeatures/

DiGraph_AllFeatures/  edges.csv  meta.yaml  nodes.csv


In [18]:
print("Versión de DGL:", dgl.__version__)
print("Versión de PyTorch:", torch.__version__)

TOR_LABELS_DICT = {'P2P':0, 'C2P': 1,'P2C': 2}

Versión de DGL: 2.4.0+cu121
Versión de PyTorch: 2.3.0+cu121


In [19]:
# data_path = "data/DGL_Graph/DiGraph_AllFeatures/"
# import pandas as pd

# # Cargar los archivos CSV
# edges = pd.read_csv("data/DGL_Graph/DiGraph_AllFeatures/edges.csv")
# nodes = pd.read_csv("data/DGL_Graph/DiGraph_AllFeatures/nodes.csv")

# # Obtener los conjuntos de nodos
# edge_nodes = set(edges['src_id']).union(set(edges['dst_id']))
# node_ids = set(nodes['node_id'])

# print(f"Total de nodos en edges.csv: {len(edge_nodes)}")
# print(f"Total de nodos en nodes.csv: {len(node_ids)}")

# # Verificar que todos los nodos en edges.csv estén presentes en nodes.csv
# missing_nodes = edge_nodes - node_ids
# if missing_nodes:
#     print(f"Faltan los siguientes nodos en nodes.csv: {missing_nodes}")
# else:
#     print("Todos los nodos en edges.csv están presentes en nodes.csv")

# print(f"Total de nodos: {len(missing_nodes)}")



# Diccionario de casos de prueba

In [20]:
cases = {
    # "CASO_1": {
    #     "data_path": "data/DGL_Graph/DiGraph_AllFeatures/",
    #     "features_type": "AllFeatures",
    #     "size_train": 0.8,
    #     "model": GCN,
    #     "predictor": DotPredictor,
    #     'sampling': None,
    #     'description': 'GCN con predictor de Dot'
    # },
    # "CASO_2": {
    #     "data_path": "data/DGL_Graph/DiGraph_AllFeatures/",
    #     "features_type": "AllFeatures",
    #     "size_train": 0.8,
    #     "model": GCN,
    #     "predictor": MLPPredictor,
    #     'sampling': None,
    #     'description': 'GCN con predictor de MLP'
    # },
    # "CASO_3": {
    #     "data_path": "data/DGL_Graph/DiGraph_AllFeatures/",
    #     "features_type": "AllFeatures",
    #     "size_train": 0.8,
    #     "model": GraphSAGE,
    #     "predictor": DotPredictor,
    #     'sampling': None,
    #     "description": "GraphSAGE con predictor de Dot"
    # },
    # "CASO_4": {
    #     "data_path": "data/DGL_Graph/DiGraph_AllFeatures/",
    #     "features_type": "AllFeatures",
    #     "size_train": 0.8,
    #     "model": GraphSAGE,
    #     "predictor": MLPPredictor,
    #     'sampling': None,
    # },
    # "CASO_5": {
    #     "data_path": "data/DGL_Graph/DiGraph_DegreeFeatures/",
    #     "features_type": "DiGraph_DegreeFeatures",
    #     "size_train": 0.8,
    #     "model": GraphSAGE,
    #     "predictor": DotPredictor,
    #     'sampling': None,
    # },
    # "CASO_6": {
    #     "data_path": "data/DGL_Graph/DiGraph_DegreeFeatures/",
    #     "features_type": "DiGraph_DegreeFeatures",
    #     "size_train": 0.8,
    #     "model": GraphSAGE,
    #     "predictor": MLPPredictor,
    #     'sampling': None,
    # },
    # "CASO_7": {
    #     "data_path": "data/DGL_Graph/DiGraph_AllFeatures/",
    #     "features_type": "DiGraph_DegreeFeatures",
    #     "size_train": 0.8,
    #     "model": ModelSAGESample,
    #     "predictor": MLPPredictor,
    #     "sampling": dgl.dataloading.NeighborSampler,
    #     "description": "Sampling con neighbour node sampling y batch normalizationGraphSAGE con predictor de Dot"
    # },
    "CASO_8": {
        "data_path": "data/DGL_Graph/DiGraph_AllFeatures/",
        "features_type": "DiGraph_DegreeFeatures",
        "size_train": 0.8,
        "model": GraphSAGE,
        "predictor": MLPPredictor,
        "sampling": dgl.dataloading.ClusterGCNSampler,
        "description": "Sampling con neighbour node sampling y batch normalizationGraphSAGE con predictor de Dot"
    },
    # "CASO_8": {
    # TODO: Agregar caso only degree y otros
}

# Funciones Auxiliares

In [28]:

def load_and_prepare_data(data_path, size_train):
    gnn = GNN(debug=True)
    gnn.load_dataset(data_path, force_reload=True)
    gnn.split_dataset(size_train)
    # gnn.split_dtaset_v2()
    gnn.dgl_graph.ndata['feat'] = gnn.dgl_graph.ndata['feat'].float() #FIXME:
    return gnn


def initialize_model(gnn, model, predictor, lr ,h_feats_model, out_feat_model , out_feat_predictor = None):
    in_feats = gnn.dgl_graph.ndata['feat'].shape[1]
    print('GGGGGGGGGGG')
    model = model(in_feats, h_feats_model, out_feat_model)
    print('GGGGGGGGGGG')

    if predictor == DotPredictor:
        pred = predictor()
        print('DP!!!!!')
    else:
        # MLP Predictor
        print('MLP!!!!!')
        pred = predictor( out_feat_model, out_feat_predictor)
    
    optimizer = Adam(itertools.chain(model.parameters(), pred.parameters()), lr=lr)
    return model, pred, optimizer

def create_dataloaders(gnn, sampler_class ,batch_size=1000):

    train_mask = gnn.train_mask
    test_mask = gnn.test_mask

    # Obtener índices de nodos de entrenamiento
    train_nids = train_mask.nonzero(as_tuple=True)[0]

    # Obtener índices de nodos de prueba
    test_nids = test_mask.nonzero(as_tuple=True)[0]
    # Definir Dataloader y Sampler Training
    
    if sampler_class == dgl.dataloading.NeighborSampler:
        print('NeighborSampler Data Loader')

        sampler_train = sampler_class([2, 2])

        # Transformar el sampler para predicción de aristas
        sampler_train = dgl.dataloading.as_edge_prediction_sampler(
            sampler_train, 
            # exclude='reverse_id',  # Excluir aristas inversas
            # reverse_eids=reverse_eids,  # Especificar las aristas inversas
            # negative_sampler=dgl.dataloading.negative_sampler.Uniform(5)  # Sampler negativo con 5 muestras negativas
        )



        

        train_dataloader = dgl.dataloading.DataLoader(
            # The following arguments are specific to DGL's DataLoader.
            gnn.dgl_graph,              # The graph
            train_nids,         # The node IDs to iterate over in minibatches
            sampler_train,            # The neighbor sampler
            # The following arguments are inherited from PyTorch DataLoader.
            batch_size=1000,    # Batch size
            shuffle=True,       # Whether to shuffle the nodes for every epoch
            drop_last=False,    # Whether to drop the last incomplete batch
            # num_workers=0       # Number of sampler processes
        )



        # Definir Dataloader y Sampler Evaluación ------------------------------------------------------------------
        sampler_test = sampler_class([2,2])


        # Transformar el sampler para predicción de aristas
        sampler_test = dgl.dataloading.as_edge_prediction_sampler(
            sampler_test, 
            # exclude='reverse_id',  # Excluir aristas inversas
            # reverse_eids=reverse_eids,  # Especificar las aristas inversas
            # negative_sampler=dgl.dataloading.negative_sampler.Uniform(5)  # Sampler negativo con 5 muestras negativas
        )

        test_dataloader = dgl.dataloading.DataLoader(
            # The following arguments are specific to DGL's DataLoader.
            gnn.dgl_graph,              # The graph
            test_nids,         # The node IDs to iterate over in minibatches
            sampler_test,            # The neighbor sampler
            # The following arguments are inherited from PyTorch DataLoader.
            batch_size=1000,    # Batch size
            shuffle=True,       # Whether to shuffle the nodes for every epoch
            drop_last=False,    # Whether to drop the last incomplete batch
            # num_workers=0       # Number of sampler processes
        )







    elif sampler_class == dgl.dataloading.ClusterGCNSampler:
        print('ClusterGCNSampler Data Loader')

        num_parts = 1000
        g = gnn.dgl_graph
        sampler = dgl.dataloading.ClusterGCNSampler(g, num_parts)
        train_dataloader = dgl.dataloading.DataLoader(
            g, 
            torch.arange(num_parts), 
            sampler,
            batch_size=20, 
            shuffle=True, 
            drop_last=False, 
            num_workers=4
        )

        test_dataloader = dgl.dataloading.DataLoader(
            g, 
            torch.arange(num_parts), 
            sampler,
            batch_size=20, 
            shuffle=True, 
            drop_last=False, 
            num_workers=4
        )

        # for subg in dataloader:
        #     print(subg)
        #     print("-------------------------------")





    return train_dataloader, test_dataloader




def train_model(case, gnn, model, pred, optimizer, epochs=100):
    train_mask = gnn.train_mask
    test_mask = gnn.test_mask
    labels = gnn.dgl_graph.edata["Relationship"].float()

    model_complexity = []
    acc_val = []
    acc_train = []
    train_error = []
    val_error = []

    for epoch in range(epochs):
        # forward
        h = model(gnn.dgl_graph, gnn.dgl_graph.ndata['feat'])
        logits = pred(gnn.dgl_graph, h)

        # Calculo pérdida
        train_loss = F.binary_cross_entropy_with_logits(logits[train_mask], labels[train_mask])
        val_loss = F.binary_cross_entropy_with_logits(logits[gnn.val_mask], labels[gnn.val_mask])

        # Calculo accuracy
        train_acc = logits[train_mask].detach().numpy()
        val_acc = logits[gnn.val_mask].detach().numpy()

        # Guardar valores
        train_error.append(train_loss.item())
        val_error.append(val_loss.item())
        acc_train.append(train_acc)
        acc_val.append(val_acc)
        model_complexity.append(epoch + 1)

        # backward
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch % 20 == 0:
            print('In epoch {}, train loss: {}, val loss: {}'.format(epoch, train_loss, val_loss))

    # Guardar los valores en un archivo CSV
    df = pd.DataFrame({
        'epoch': model_complexity,
        'train_error': train_error,
        'val_error': val_error,
        'acc_train': acc_train,
        'acc_val': acc_val
    })
    df.to_csv(f'results/binary_classification/training_results_{case}.csv', index=False)

    return logits, train_error, val_error, acc_train, acc_val, model_complexity


def train_model_sampler_neighbour(case, gnn, model, pred, optimizer,train_dataloader,test_dataloader, epochs=100):

    acc_val = []
    acc_train = []

    train_error = []
    train_error_per_epoch = []
    val_error_per_epoch = []
    train_edges_id = []

    # Inicializar listas para guardar logits y labels del entrenamiento
    all_logits_train = []
    all_labels_train = []

    # Inicializar listas para guardar logits y labels de la validacion
    all_logits_val = []
    all_labels_val = []
    # Almacenar la importancia de características por epoch
    feature_importances = []
    for epoch in range(50):
        model.train()
        val_loss_epoch=0
        num_batches = 0

        with tqdm.tqdm(train_dataloader) as tq:
            for step, (input_nodes, output_graph, mfgs) in enumerate(tq):
        
                mfgs = [graph for graph in mfgs]
                edge_ids = output_graph.edata[dgl.EID]
                inputs = mfgs[0].srcdata['feat']
                # Hacer que los inputs requieran gradiente
                # Embeddings de nodos
                h = model(mfgs, inputs)
                logits =  pred(output_graph, h)
            
                
                
                train_edges_id.append(edge_ids)

                # Usar los IDs de los bordes para obtener las etiquetas reales
                labels_batch_output = gnn.dgl_graph.edata['Relationship'][edge_ids].to(torch.int64)
                
                all_logits_train.append(logits.detach().cpu().numpy())
                all_labels_train.append(labels_batch_output.cpu().numpy())

                # Calculo pérdida --------------------------------------------------------
                train_loss = F.cross_entropy(logits, labels_batch_output)
                train_error.append(train_loss.item())

                val_loss_epoch += train_loss.item()
                num_batches += 1


                # val_loss_epoch += train_loss.item()
                # num_batches += 1
                
                # backward
                optimizer.zero_grad()
                train_loss.backward()
                optimizer.step()
        train_error_per_epoch.append(val_loss_epoch / num_batches)


        # Validación
        model.eval()
        val_loss_epoch = 0
        num_batches = 0
        with torch.no_grad():
            with tqdm.tqdm(test_dataloader) as tq:
                for step, (input_nodes, output_graph, mfgs) in enumerate(tq):
                    mfgs = [graph for graph in mfgs]
                    edge_ids = output_graph.edata[dgl.EID]
                    inputs = mfgs[0].srcdata['feat']
                    
                    
                    # Forward para validación
                    h_val = model(mfgs, inputs)
                    logits_val = pred(output_graph, h_val)

                    # Guardar logits de validacion
                    all_logits_val.append(logits_val.detach().cpu().numpy())

                    # Obtener las etiuetas reales
                    labels_batch_output_val = gnn.dgl_graph.edata['Relationship'][edge_ids].to(torch.int64)
                    all_labels_val.append(labels_batch_output_val.cpu().numpy())
                
                    # Calcular pérdida de validación
                    val_loss_batch = F.cross_entropy(logits_val, labels_batch_output_val)
                    val_loss_epoch += val_loss_batch.item()
                    num_batches += 1
                # Almacenar el promedio de la pérdida de validación por epoch
        val_error_per_epoch.append(val_loss_epoch / num_batches)
        if epoch % 1 == 0:
            print('In epoch {}, train loss: {},'.format( epoch,train_loss))



def train_model_sampler_cluster(case,model, pred, optimizer,train_dataloader,test_dataloader, epochs=100):
    # ----------- 4. training -------------------------------- #
    acc_val = []
    acc_train = []

    train_error = []
    train_error_per_epoch = []
    val_error_per_epoch = []
    train_edges_id = []

    # Inicializar listas para guardar logits y labels del entrenamiento
    all_logits_train = []
    all_labels_train = []

    # Inicializar listas para guardar logits y labels de la validacion
    all_logits_val = []
    all_labels_val = []
    
    for epoch in range(epochs):
        # model.train()
        for it, sg in enumerate(train_dataloader):
            # print(it,sg)
            inputs = sg.ndata['feat'].float()
            # print("[INPUTS]",inputs)
            labels = sg.edata['Relationship'].float()

            train_mask = sg.edata['train_mask'].bool()
            # print("[TRAIN_MASK]",train_mask)

            h = model(sg, inputs)
            logits =  pred(sg, h)

            # print("[LABELS]",labels)
            all_logits_train.append(logits[train_mask].detach().cpu().numpy())
            all_labels_train.append(labels[train_mask].cpu().numpy())
            
            # logits = model(sg, inputs)
            train_loss = F.binary_cross_entropy_with_logits(logits[train_mask], labels[train_mask])
            train_error.append(train_loss.item())
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            # if epoch % 5000 == 0:
            #     print(f"Loss {train_loss}")

            


            model.eval()
            with torch.no_grad():
                val_preds, test_preds = [], []
                val_labels, test_labels = [], []
                for it, sg in enumerate(test_dataloader):
                    inputs = sg.ndata["feat"]
                    labels = sg.edata["Relationship"]

                    val_mask = sg.edata["val_mask"].bool()
                    test_mask = sg.edata["test_mask"].bool()

                    h = model(sg, inputs)
                    logits =  pred(sg, h)

                    val_preds.append(logits[val_mask])
                    val_labels.append(labels[val_mask])
                    test_preds.append(logits[test_mask])
                    test_labels.append(labels[test_mask])

                val_preds = torch.cat(val_preds, 0)
                val_labels = torch.cat(val_labels, 0)
                test_preds = torch.cat(test_preds, 0)
                test_labels = torch.cat(test_labels, 0)

                all_logits_val.append(val_preds)
                all_labels_val.append(val_labels)

    # Concatenar todos los logits y etiquetas al final del entrenamiento
    all_logits_train = np.concatenate(all_logits_train, axis=0)
    all_labels_train = np.concatenate(all_labels_train, axis=0)
    all_logits_val = np.concatenate(all_logits_val, axis=0)
    all_labels_val = np.concatenate(all_labels_val, axis=0)

    # Crear el directorio si no existe
    os.makedirs('results/binary_classification', exist_ok=True)

    # Guardar los valores en archivos CSV
    df_train = pd.DataFrame({
        'logits': all_logits_train.flatten(),
        'labels': all_labels_train.flatten()
    })
    df_train.to_csv(f'results/binary_classification/logits_labels_train_{case}.csv', index=False)

    df_val = pd.DataFrame({
        'logits': all_logits_val.flatten(),
        'labels': all_labels_val.flatten()
    })
    df_val.to_csv(f'results/binary_classification/logits_labels_val_{case}.csv', index=False)


    return all_logits_train, all_labels_train, train_error, all_logits_val, all_labels_val
def evaluate_model(gnn, all_logits_train, train_mask_or_labels ,sampling = None):
    
    if not sampling:
        train_mask = train_mask_or_labels
        true_train_labels = gnn.dgl_graph.edata["Relationship"][train_mask]
        predicted_train_scores = all_logits_train.detach().numpy()[train_mask]
    else:
        # Caso sampling
        print('Evaluando modelo con sampling')
        print('train_mask_or_labels',len(train_mask_or_labels))
        print('all_logits_train',len(all_logits_train))
        true_train_labels = train_mask_or_labels
        predicted_train_scores = all_logits_train

    optimal_threshold = plot_roc_curve(true_train_labels, predicted_train_scores)
    return optimal_threshold

def plot_error_over_epoch(train_error, train_error_per_epoch, val_error_per_epoch):
    # Crear gráfico del error de entrenamiento
    plt.figure()  # Crear una nueva figura
    plt.plot([i for i in range(len(train_error))],train_error, label='Train Error', color='blue')  # Error de entrenamiento en azul
    plt.xlabel('Mini-Batch')
    plt.ylabel('Error')
    plt.title('Training Error over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()  # Mostrar el gráfico de error de entrenamiento


    # Crear gráfico combinado de error de entrenamiento y validación
    plt.figure()  # Crear una nueva figura
    plt.plot(train_error_per_epoch, label='Train Error', color='blue')  # Error de entrenamiento en azul
    plt.plot(val_error_per_epoch, label='Validation Error', color='orange')  # Error de validación en naranja
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.title('Training and Validation Error over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()  # Mostrar el gráfico combinado



# Entrenamiento 

In [29]:
for case_name, case in cases.items():
    print(f"[{case_name}]")

    data_path = case["data_path"]
    size_train = case["size_train"]

    # ----------- 1. Cargar grafo y preparar datos -------------- #
    gnn = load_and_prepare_data(data_path, size_train)

    if case['sampling']: 
            sampler_class = case['sampling']
            train_dataloader, test_dataloader = create_dataloaders(gnn,sampler_class,batch_size=1000)

            print('train_dataloader',train_dataloader)
            print('test_dataloader',test_dataloader)
    # ----------- 2. Inicializa modelo -------------------------- #
    model = case["model"]
    predictor = case["predictor"]
    lr = 0.01
    if predictor == DotPredictor:
        print("DotPredictor")
        h_feats_model = 16
        out_feat_model = 2 # num clases
        model, pred, optimizer = initialize_model(gnn, model, predictor, lr, h_feats_model, out_feat_model)
    else:
        print("MLP")
        h_feats_model = 16
        out_feat_model = 32
        out_feat_predictor = 1 # num clases
        model, pred, optimizer = initialize_model(gnn, model, predictor, lr ,h_feats_model, out_feat_model , out_feat_predictor)


    # ----------- 3. Entrenar modelo ----------------------------- #
    epochs = 2
    
    # if case['sampling'] == dgl.dataloading.NeighborSampler:
    #     all_logits_train, all_labels_train, train_error_per_epoch, val_error_per_epoch,train_error,all_logits_val,all_labels_val =train_model_sampler_neighbour(case_name, gnn, model, pred, optimizer,train_dataloader,test_dataloader, epochs)
    
    # elif case['sampling'] == dgl.dataloading.ClusterGCNSampler:
    #     all_logits_train, all_labels_train, train_error, all_logits_val, all_labels_val = train_model_sampler_cluster(case_name,model, pred, optimizer,train_dataloader,test_dataloader, epochs=100)

    # else:
    #     all_logits_train, train_error, val_error, acc_train, acc_val, model_complexity = train_model(case_name, gnn, model, pred, optimizer, epochs)

    if case['sampling'] == dgl.dataloading.NeighborSampler:
        print('CASO NEIGHBOUR')
        train_model_sampler_neighbour(case_name, gnn, model, pred, optimizer,train_dataloader,test_dataloader, epochs)
    
    elif case['sampling'] == dgl.dataloading.ClusterGCNSampler:
        print('CASO CLUSTER')
        train_model_sampler_cluster(case_name,model, pred, optimizer,train_dataloader,test_dataloader, epochs)
    

    # # ----------- 4. Definir threashold ----------------------------- #
    
    # if case['sampling']:
    #     optimal_threshold = evaluate_model(gnn, all_logits_train, all_labels_train,case['sampling'])
    #     plot_error_over_epoch(train_error, train_error_per_epoch, val_error_per_epoch)
    # else:
    #     optimal_threshold = evaluate_model(gnn, all_logits_train, gnn.train_mask,case['sampling'])

    # print(f"Optimal Threshold: {optimal_threshold}")

    # # ----------- 5. Plotear resultados ----------------------------- #
    # if  case['sampling']:
    #     # FIXME: ver que son los resultados
    #     calculate_metrics(all_logits_val,all_labels_val , optimal_threshold)
    # else:
    #     plot_training(case_name,gnn,train_error_per_epoch,acc_train,val_error_per_epoch,acc_val,model_complexity,optimal_threshold)
    # # plot_training(case_name,gnn,train_error,acc_train,val_error,acc_val,model_complexity,optimal_threshold)

    # # print("------------------------------------------------------")
    # # print("------------------------------------------------------")

[CASO_8]
Done saving data into cached files.
Graph(num_nodes=74145, num_edges=461889,
      ndata_schemes={'feat': Scheme(shape=(72,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.int64)})
Training edges: 369736
Validation edges: 46040
Test edges: 46113
ClusterGCNSampler Data Loader
train_dataloader <dgl.dataloading.dataloader.DataLoader object at 0x761dbcfb9390>
test_dataloader <dgl.dataloading.dataloader.DataLoader object at 0x761dbcfb9ab0>
MLP
GGGGGGGGGGG
GGGGGGGGGGG
MLP!!!!!
CASO CLUSTER


/home/valentina/Desktop/GIT/TrabajoTesis/env310/lib/python3.10/site-packages/dgl/dataloading/dataloader.py:1144: DGLWarning: Dataloader CPU affinity opt is not enabled, consider switching it on (see enable_cpu_affinity() or CPU best practices for DGL [https://docs.dgl.ai/tutorials/cpu/cpu_best_practises.html])
  dgl_warning(
